<a href="https://colab.research.google.com/github/giswqs/GEE-Courses/blob/master/docs/gee_intro/Analysis/zonal_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
pip install --upgrade geemap


In [2]:
import ee
import geemap
import os

In [3]:
ee.Authenticate()

True

In [4]:
ee.Initialize(project="ee-kipronopatrick273")

## Create an interactive map

## Add Earth Engine data

In [5]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
Map = geemap.Map()
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add Landsat data to map
landsat = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

#Load Kenyan counties
# Load Kenya counties from GAUL Level 2
counties = ee.FeatureCollection("FAO/GAUL/2015/level2") \
                     .filter(ee.Filter.eq('ADM0_NAME', 'Kenya'))

# Filter for Uasin Gishu county
uasin_gishu = counties.filter(ee.Filter.eq('ADM2_NAME', 'Uasin Gishu'))


Map.centerObject(uasin_gishu, 8);
Map.addLayer(uasin_gishu, {}, 'uasin_gishu')
Map







Map(center=[0.534914996597277, 35.319908156209], controls=(WidgetControl(options=['position', 'transparent_bg'…

## Compute zonal statistics for one image

In [6]:
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [8]:
out_dem_stats = os.path.join(out_dir, 'dem_stats.csv')

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, counties, out_dem_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/dem_stats.csv


In [ ]:
out_landsat_stats = os.path.join(out_dir, 'landsat_stats.csv')
geemap.zonal_statistics(
    landsat, kenya, out_landsat_stats, statistics_type='SUM', scale=1000
)

## Compute zonal statistics for time-series images

In [ ]:
# Load and prepare NDVI collection
ndvi = ee.ImageCollection("MODIS/MCD43A4_006_NDVI") \
          .filterBounds(uasin_gishu) \
          .filterDate('2018-01-01', '2022-12-31') \
          .select('NDVI') \
          .map(lambda img: img.multiply(0.0001).copyProperties(img, ['system:time_start']))

# Use the median image over the period
ndvi_median = ndvi.median().clip(uasin_gishu)

# Green color palette
green_palette = {
    'min': 0.0,
    'max': 1.0,
    'palette': [
        'f7fcf5', 'e5f5e0', 'c7e9c0',
        'a1d99b', '74c476', '41ab5d',
        '238b45', '006d2c', '00441b'
    ]
}

# Display on map
Map = geemap.Map(center=[0.45, 35.3], zoom=8)
Map.addLayer(ndvi_median, green_palette, "NDVI (2018–2022 Median)")
Map.addLayer(uasin_gishu, {}, "Uasin Gishu Boundary")
Map

In [ ]:
# Create visualization parameters
vis_params = {
    'min': 0.0,
    'max': 0.8,
    'palette': [
        'f7fcf5', 'e5f5e0', 'c7e9c0',
        'a1d99b', '74c476', '41ab5d',
        '238b45', '006d2c', '00441b'
    ]
}

# Output GIF settings
video_args = {
    'dimensions': 600,
    'region': uasin_gishu.geometry(),
    'framesPerSecond': 2,
    'format': 'gif',
    'min': 0.0,
    'max': 0.8,
    'palette': vis_params['palette']
}

# Export GIF
gif_url = geemap.get_image_collection_gif(ndvi_col, video_args)
geemap.download_ee_video(ndvi_col, video_args, 'ndvi_uasin_gishu.gif')

# Preview (in notebook)
geemap.show_image('ndvi_uasin_gishu.gif')

In [ ]:
modis = collection.toBands()
Map.addLayer(modis, {}, "MODIS Time series", False)

In [ ]:
out_landsat_stats = os.path.join(out_dir, 'ndvi.csv')
geemap.zonal_statistics(
    modis, states, out_landsat_stats, statistics_type='MEAN', scale=1000
)